In [ ]:
import pandas as pd
import numpy as np
import os

from datetime import datetime, timedelta

In [ ]:
filename = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Current_BSFS_Barnn_Growth.csv'
df = pd.read_csv(filename, sep = ',')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
df.columns = df.columns.str.replace('#', 'num')
df.columns = df.columns.str.replace('%', 'PER')

# df['This_Year_Growth_Goal'] = df['This_Year_Growth_Goal'].str.replace(',','')

filename2 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/BSFS_BARNN_Growth_Incentive.txt'
df_tierMap = pd.read_csv(filename2, sep = '\t')
df_tierMap.columns = df_tierMap.columns.str.replace(' ', '_')

df.shape

In [ ]:
# df_tierMap

In [ ]:
# df.columns.to_series(index=range(df.shape[1])) 

In [ ]:
df_select = df.iloc[:,[0,1,67,68,69,70,71,72,73,74,75,76,77,78,79,95]]
df_select = df_select.assign(YTD_MB_Sum=0,CurrentTier=0,CreateDate=0,tierUP=1,CurrentPayout=0)

df_select['YTD_MB_Sum'] = df_select.H1_MB + df_select.H1_MB_2nd_

# df_select.head(2)

In [ ]:
currentYear = datetime.now().year

In [ ]:
tup_now = datetime.now().timetuple()

In [ ]:
ts_currentData = os.path.getctime(filename)
dt_currentData = datetime.fromtimestamp(ts_currentData) - timedelta(days=35) # USED FOR TESTING-POC *** THIS SHOULD BE CHANGED
tup_currentData = dt_currentData.timetuple()
print(tup_currentData)
df_select['CreateDate'] = str(tup_currentData.tm_year) + '-' + str(tup_currentData.tm_mon) + '-' + str(tup_currentData.tm_mday)

ts_incentive = os.path.getctime(filename2)
dt_incentive = datetime.fromtimestamp(ts_incentive)
# print(dt_currentData) 

In [ ]:
# tup_currentData

In [ ]:
# df_select.head()

In [ ]:
# df_select2 = df_select[
#     ['Dealer_Name_','H1_BS', 'H1_FS', 'H1_MB', 'H1_FZ', 'H1_TOTAL', 'H1_BS_2nd_',
#    'H1_FS_2nd_', 'H1_MB_2nd_', 'H1_FZ_2nd_', 'H1_Total_2nd',
#    'H1_TOTAL_DEL', 'H1_TOTAL_NET', 'H1_PER_MB','This_Year_Growth_Goal',
#    'YTD_MB_Sum','CurrentTier','CreateDate','tierUP','CurrentPayout']]

In [ ]:
calculatedData = pd.DataFrame([])
for ii in range(df.shape[0]):

    singleCusty = df_select.iloc[[ii]]
    singleCusty.reset_index(inplace=True)

    if  tup_currentData.tm_year == currentYear:

#         if (tup_currentData.tm_mon == 1) | (tup_currentData.tm_mon == 2) | (tup_currentData.tm_mon == 3) | (tup_currentData.tm_mon == 4) | (tup_currentData.tm_mon == 5) | (tup_currentData.tm_mon == 6):
        if tup_currentData.tm_mon in [1,2,3,4,5,6]:    
            if ((tup_currentData.tm_year % 400 == 0) | (tup_currentData.tm_year % 4 == 0)) & (tup_currentData.tm_year % 100 != 0):
                half = 1
                daysINhalf = 182
                print('Leap year')
                df_select.loc[df_select['YTD_MB_Sum'] < (df_select.This_Year_Growth_Goal/2), 'CurrentTier'] = 0
                df_select.loc[df_select['YTD_MB_Sum'] >= (df_select.This_Year_Growth_Goal/2), 'CurrentTier'] = 1
            else:
                half = 1
                daysINhalf = 181
                df_select.loc[df_select['YTD_MB_Sum'] < (df_select.This_Year_Growth_Goal/2), 'CurrentTier'] = 0
                df_select.loc[df_select['YTD_MB_Sum'] >= (df_select.This_Year_Growth_Goal/2), 'CurrentTier'] = 1
        else:
            half = 2
            daysINhalf = 365
            df_select.loc[df_select['YTD_MB_Sum'] < (df_select.This_Year_Growth_Goal), 'CurrentTier'] = 0
            df_select.loc[df_select['YTD_MB_Sum'] >= (df_select.This_Year_Growth_Goal), 'CurrentTier'] = 1
            print(half)     
            
        if singleCusty.CurrentTier.values > 0:
            singleCusty.CurrentPayout = singleCusty.YTD_MB_Sum.values * df_tierMap.iloc[1,1]
        else:
            singleCusty.CurrentPayout = 0
            
        numDays = tup_currentData.tm_yday # get number of days

        estimatedTires = np.floor(singleCusty.YTD_MB_Sum.astype(int)/(numDays/daysINhalf)).values
        
        # calculates est tier
        if half == 1:
            if estimatedTires < (singleCusty.This_Year_Growth_Goal/2).values:
                est_Tier = 0
            else: 
                est_Tier = 1
        elif half == 2:
            if estimatedTires < (singleCusty.This_Year_Growth_Goal).values:
                est_Tier = 0
            else: 
                est_Tier = 1
                            
        tierUP = 1        
        barnn_growth_est_rebate = estimatedTires * df_tierMap.iloc[est_Tier,1]
        
        #1Up Values
        if half == 1:
            if est_Tier == 0:
                barnn_growth_1up_tires =  (singleCusty.This_Year_Growth_Goal/2) - estimatedTires
                barnn_growth_1up_tires_rebate = (singleCusty.This_Year_Growth_Goal/2) * df_tierMap.iloc[tierUP,1]
            else:
                barnn_growth_1up_tires = 0
                barnn_growth_1up_tires_rebate = barnn_growth_est_rebate
        else:
            if est_Tier == 0:
                barnn_growth_1up_tires =  (singleCusty.This_Year_Growth_Goal) - estimatedTires
                barnn_growth_1up_tires_rebate = (singleCusty.This_Year_Growth_Goal) * df_tierMap.iloc[tierUP,1]
            else:
                barnn_growth_1up_tires = estimatedTires
                barnn_growth_1up_tires_rebate = barnn_growth_est_rebate


        range_dollars_ll = ((barnn_growth_1up_tires*df_tierMap.iloc[0,1]))#.values
        range_dollars_ul = ((barnn_growth_1up_tires*df_tierMap.iloc[tierUP,1]))#.values
        
        ###########################################################################################################################        
#         print(singleCusty_select)
# ###########################################################################################################################     
#         print('\nEstimated Tier:', [estTier])        
#         print('\nEstimated Tires ~ avi:', estimatedTires)        
#         print('\nEstimated Rebate ~ avi:',barnn_growth_est_rebate)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################
#         print('Estimated 1up Tier:', [tierUP])        
#         print('\n1up Tires ~ avi:', barnn_growth_1up_tires)        
#         print('\n1up Dollars ~ avi:',barnn_growth_1up_tires_rebate)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################        
#         print('Range_ll:',range_dollars_ll)
#         print('Range_ul:',range_dollars_ul, '\n')
# ###########################################################################################################################
#         print(tt, '\n' 'indexNumber:', ii, '\n')
###########################################################################################################################

        tempData = pd.DataFrame(
            {'estimated_tier':est_Tier,
             'current_est_tires':estimatedTires, 'total_est_rebate':barnn_growth_est_rebate, 'period':half, 'grain': 'Semi-Annual',
             'Up_tier':tierUP,
             'barnn_growth_up_tires':barnn_growth_1up_tires, 'barnn_growth_up_rebate':barnn_growth_1up_tires_rebate,
             'range_lower':range_dollars_ll, 'range_upper':range_dollars_ul})
        calculatedData = calculatedData.append(tempData)
    else:
#         print('\nJUNK:', tt,'\n', 'indexNumber:', ii)
        tempData = pd.DataFrame(
            {'estimated_tier':0,
             'current_est_tires':0, 'total_est_rebate':0, 'period':half, 'grain': 'Semi-Annual',
             'Up_tier':1,
             'barnn_growth_up_tires':0, 'barnn_growth_up_rebate':0,
             'range_lower':range_dollars_ll, 'range_upper':range_dollars_ul})
        calculatedData = calculatedData.append(tempData)
#         print(ii)

###########################################################################################################################        
calculatedData = calculatedData.reset_index(drop=True)
finalData = pd.concat([calculatedData,df_select], axis=1, sort=True)

In [ ]:
finalData = finalData.replace([np.inf, -np.inf], np.nan).fillna(0)
# finalData.head()

In [ ]:
path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
writeFilename = path + 'CMP_BSFS_BARNN_GROWTH.csv'
finalData.to_csv(writeFilename, index=False)
writeFilename_archive = path + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_BSFS_BARNN_GROWTH_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
finalData.to_csv(writeFilename_archive, index=False)

In [ ]:
# path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
# writeFilename = path + 'cmp_G3X.csv'
# finalData.to_csv(writeFilename, index=False)
# writeFilename_archive = path + 'csv_archive/' + str(dt_currentData.year) + '_' + str(dt_currentData.month) + '_' + str(dt_currentData.day) + '_' + 'CMP_G3X.csv'
# finalData.to_csv(writeFilename_archive, index=False)